In [2]:
import warnings
# External Imports
from IPython.display import display, clear_output
from tkinter import Tk,filedialog
# import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox, Layout, DatePicker, HTML, RadioButtons, SelectMultiple, Output, Checkbox
import os
import datetime
import glob
import json
import pandas as pd
import geopandas as gpd

# New imports for coastsat shoreline extraction
import numpy as np
# Local Imports 
from CoastSeg import download_roi, file_functions, zoo_model_module, coastseg_map_class
from CoastSeg.CoastSat.coastsat import SDS_shoreline,SDS_download
# suppress tensorflow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
root = Tk()

Version:  2.9.1
Eager mode:  True
Version:  2.9.1
Eager mode:  True
GPU name:  []
Num GPUs Available:  0


In [3]:
# Date Widgets
start_date=DatePicker(
    description='Start Date',
    value=datetime.date(2018, 12, 1),
    disabled=False,
)
end_date=DatePicker(
    description='End Date',
    value=datetime.date(2019, 3, 1), #2019, 1, 1
    disabled=False,
)
date_instr=HTML(
    value="<b>Pick a date:</b>",
    layout=Layout(padding='10px')
)
dates_box=HBox([start_date,end_date])
dates_vbox=VBox([date_instr,dates_box])


collection_instr=HTML(
    value="<b>Pick a collection</b>\
        <br> C01: LandSat Collection 1 \
        <br> C02: LandSat Collection 2  [2022/01/01 and beyond] \
        <br>      - Landsat 9 is only available in C02",
    layout=Layout(padding='10px')
)
collection_radio=RadioButtons(
    options=['C01', 'C02'],
    value='C01',
    description='Collection:',
    disabled=False
)
collection_vbox=VBox([collection_instr,collection_radio])

sat_instr=HTML(
    value="<b>Pick multiple satellites by holding the control key:</b> \
        <br> - images prior to 2022/01/01 will be downloaded from Collection 1 \
        <br> - images after 2022/01/01 will be automatically downloaded from Collection 2 \
        <br> C01: LandSat Collection 1 : All dates before 2022/01/01\
        <br> C02: LandSat Collection 2 : All dates after 2022/01/01 \
        <br> - Landsat 9 is only available in C02",
    layout=Layout(padding='10px')
)


satellite_selection=SelectMultiple(
    options=['L5', 'L7', 'L8', 'S2'],
    value=['L8'],
    description='Satellites',
    disabled=False
)
sat_vbox = VBox([sat_instr,satellite_selection])

def handle_collection_change(change):
    if change['new'] == 'C02':
        satellite_selection.options=['L5', 'L7', 'L8', 'L9', 'S2']
    else:
        satellite_selection.options=['L5', 'L7', 'L8', 'S2']

collection_radio.observe(handle_collection_change,"value")

controls_vbox=VBox([dates_vbox, collection_vbox, sat_vbox])
controls_vbox

In [4]:
# CoastSat Download Variables
# ------------------------------
# Save the satellites selected to sat_list
if satellite_selection.value:
    sat_list = list(satellite_selection.value)
elif not satellite_selection.value:
    print("ERROR: You must select at least one satellite first")
# Save the dates selected by the user as well as the selected collection
dates = [str(start_date.value),str(end_date.value)]
collection = collection_radio.value
print(f"dates: {dates}")
print(f"collection: {collection}")
print(f"sat_list: {sat_list}")

dates: ['2018-12-01', '2019-03-01']
collection: C01
sat_list: ['L8']


In [5]:
pre_process_settings = { 
    # general parameters:
    'cloud_thresh': 0.5,        # threshold on maximum cloud cover
     'dist_clouds': 300,        # ditance around clouds where shoreline can't be mapped
    'output_epsg': 3857,        # epsg code of spatial reference system desired for the output   
    # quality control:
    'check_detection': True,    # if True, shows each shoreline detection to the user for validation
    'adjust_detection': False,  # if True, allows user to adjust the postion of each shoreline by changing the threhold
    'save_figure': True,        # if True, saves a figure showing the mapped shoreline for each image
    # [ONLY FOR ADVANCED USERS] shoreline detection parameters:
    'min_beach_area': 4500,     # minimum area (in metres^2) for an object to be labelled as a beach
    'buffer_size': 150,         # radius (in metres) of the buffer around sandy pixels considered in the shoreline detection
    'min_length_sl': 200,       # minimum length (in metres) of shoreline perimeter to be valid
    'cloud_mask_issue': False,  # switch this parameter to True if sand pixels are masked (in black) on many images  
    'sand_color': 'default',    # 'default', 'dark' (for grey/black sand beaches) or 'bright' (for white sand beaches)
    'pan_off':'False',          # if True, no pan-sharpening is performed on Landsat 7,8 and 9 imagery
}

# Output wdiget used to print messages and exceptions created by CoastSeg_Map
debug_view = Output(layout={'border': '1px solid black'})

# Create the Map
- Create the map using the map settings set above

In [ ]:
coastseg_map=coastseg_map_class.CoastSeg_Map()

# How to Use The Map
---
1. Use the rectangle tool to draw a bounding box along the coastline.
2. Click `Generate ROI` to create ROI rectangles along the coastline in the bounding box. This may take some time.
- You should see a coastline appear in yellow and some rectangles along it.
3. Click 1 or more ROIs then click `Save ROI` to save these ROIs for downloading later
4. You're ready to download the ROIs. Run the next block of code.

In [ ]:
transects_button = Button(description="Load Transects")
shoreline_button = Button(description="Load Shoreline")
gen_button =Button(description="Generate ROI")
save_button = Button(description="Save ROI")
clear_debug_button = Button(description="Clear TextBox")

remove_all_button = Button(description="Remove all")
remove_transects_button = Button(description="Remove transects")
remove_bbox_button = Button(description="Remove bbox")
remove_coastline_button = Button(description="Remove coastline")
remove_rois_button = Button(description="Remove ROIs")
remove_saved_roi_button = Button(description="Remove saved ROIs")

@debug_view.capture(clear_output=True)
def on_shoreline_button_clicked(b):
    if coastseg_map.shapes_list == [] :
        print("Draw a bounding box on the coast first, then click Load Transects.")
    else:
        debug_view.clear_output(wait=True)
        print("Loading shoreline please wait.") 
        # Add the transects to the map
        coastseg_map.load_shoreline_on_map()
        print("Shoreline loaded.")

@debug_view.capture(clear_output=True)
def on_transects_button_clicked(b):
    if coastseg_map.shapes_list == [] :
        print("Draw a bounding box on the coast first, then click Load Transects.")
    else:
        debug_view.clear_output(wait=True)
        print("Loading transects please wait.") 
        # Add the transects to the map
        coastseg_map.load_transects_on_map()
        print("Transects Loaded.")

        
@debug_view.capture(clear_output=True)
def on_gen_button_clicked(b):
    if coastseg_map.shapes_list == [] :
        print("Draw a bounding box on the coast first, then click Generate ROI.")
    else:
        debug_view.clear_output(wait=True)
        print("Generating ROIs please wait.") 
        # Generate ROIs along the coastline within the bounding box
        coastseg_map.generate_ROIS_fishnet(5000,0)
        debug_view.clear_output(wait=True)
        # Add the Clickable ROIs to the map
        coastseg_map.add_geojson_layer_to_map()
        print("ROIs generated. Please Select at least one ROI and click Save ROI.")

@debug_view.capture(clear_output=True)
def on_save_button_clicked(b):
    if coastseg_map.selected_set:
        if len(coastseg_map.selected_set) == 0:
             print("Must select at least 1 ROI first before you can save ROIs.")
        else:
            debug_view.clear_output(wait=True)
            coastseg_map.save_roi_fishnet("fishnet_rois.geojson")
            print("Saving ROIs")
            debug_view.clear_output(wait=True)
            print("ROIs have been saved. Now click Download ROI to download the ROIs using CoastSat")
    else:
        print("No ROIs were selected.")


def remove_all_from_map(b):
    coastseg_map.remove_all()
def remove_transects(b):
    coastseg_map.remove_transects()
def remove_bbox_from_map(b):
    coastseg_map.remove_bbox()
def remove_coastline_from_map(b):
    coastseg_map.remove_shoreline()
def remove_all_rois_from_map(b):
    coastseg_map.remove_all_rois()
def remove_saved_roi_clicked(b):
    coastseg_map.remove_saved_roi()
def clear_debug_view(b):
    with debug_view:
        clear_output()
        

transects_button.on_click(on_transects_button_clicked)    
shoreline_button.on_click(on_shoreline_button_clicked) 
gen_button.on_click(on_gen_button_clicked)
save_button.on_click(on_save_button_clicked)
clear_debug_button.on_click(clear_debug_view)


remove_all_button.on_click(remove_all_from_map)
remove_transects_button.on_click(remove_transects)
remove_bbox_button.on_click(remove_bbox_from_map)
remove_coastline_button.on_click(remove_coastline_from_map)
remove_rois_button.on_click(remove_all_rois_from_map)
remove_saved_roi_button.on_click(remove_saved_roi_clicked)

erase_buttons=HBox([remove_all_button, remove_transects_button, remove_bbox_button, remove_coastline_button, remove_rois_button, remove_saved_roi_button])

display(debug_view,HBox([transects_button,shoreline_button, gen_button, save_button,clear_debug_button]),erase_buttons)
# Display the map
coastseg_map.m

## Download the Selected ROIs
- Download the satellite imagery for the ROIs using CoastSat.

⚠️ Make sure you have clicked `Save ROI` before running the following code.
 
### Parameters for download_imagery()
1. `coastseg_map.selected_ROI` : geojson for the ROI clicked
2. `pre_process_settings`: settings for downloading data using CoastSat
3. `dates` : date range to download the imagery
4. `sat_list` : list of satellites to download imagery from
5. `collection` : LandSat Collection 1 (`C01`) or Collection 2 (`C02`)

In [ ]:
if coastseg_map.selected_ROI:
    try:
        download_roi.download_imagery(coastseg_map.selected_ROI,pre_process_settings,dates,sat_list,collection)
    except ee.EEException as exception:
        ee.Authenticate()
        ee.Initialize()
        download_roi.download_imagery(coastseg_map.selected_ROI,pre_process_settings,dates,sat_list,collection)
else:
    print("No ROIs were selected. \nPlease select at least one ROI and click 'Save ROI' to save these ROI for download.")

In [9]:
model_dict={"sample_direc":None,"use_GPU":False, 'implementation':'ENSEMBLE','model_type':'landsat_6229071'}

model_view = Output(layout={'border': '1px solid black'})
run_model_view = Output(layout={'border': '1px solid black'})
GPU_view = Output()

run_model_button = Button(description="Run Model")
use_data_button = Button(description="Use Data Button")
use_select_images_button = Button(description="Select Your Images")


model_implementation = RadioButtons(
    options=['ENSEMBLE', 'BEST'],
   value='ENSEMBLE',
    description='Select:',
    disabled=False
)

model_dropdown = RadioButtons(
    options= ['landsat_6229071', 'landsat_6230083','SWED-RGB_6824384','SWED-MNDWI_6824342','coast-train-RGB_6950479', 'S2-water-SWED_6950474'],
    value='landsat_6229071',
    description='Select Model:',
    disabled=False,
)


GPU_checkbox = Checkbox(
    value=False,
    description='Use GPU?',
    disabled=False,
    indent=False
)


def handle_model_type(change):
    model_dict["model_type"]=change['new']


def handle_model_implementation(change):
    model_dict["implementation"]=change['new']


@GPU_view.capture(clear_output=True)
def handle_GPU_checkbox(change):
    if change["new"] == True:
        model_dict["use_GPU"]=True
        print("Using the GPU")
    else:
        model_dict["use_GPU"]=False
        print("Not using the GPU")
        

@model_view.capture(clear_output=True)
def use_data_button_clicked(button):
    # Use the data folder as the input for segmentation
    print("Loading in the jpgs from the data directory")
    # Copy the jpgs from data to a new folder called segmentation_data_[datetime]
    if 'MNDWI' in model_dropdown.value:
        sample_direc =r'C:\1_USGS\CoastSeg\repos\2_CoastSeg\CoastSeg_fork\Seg2Map\MNDWI_outputs\MNDWI_ouputs_2022-07-21__07_hr_57_min14'
#         sample_direc = file_functions.get_jpgs_from_data('MNDWI')
#         RGB_path=sample_direc+os.sep+'RGB'
#         NIR_path=sample_direc+os.sep+'NIR'
#         sample_direc = coastseg_map.RGB_to_MNDWI(RGB_path,NIR_path,sample_direc)
        model_dict["sample_direc"]=sample_direc
    else:
        sample_direc = file_functions.get_jpgs_from_data('RGB')
        model_dict["sample_direc"]=sample_direc
        jpgs = glob.glob1(sample_direc + os.sep, "*jpg")
        if jpgs == []:
            raise Exception("\nThe directory contains no jpgs! Please select a directory with jpgs.")
    print(f"\nContents of the data directory saved in {sample_direc}")


@model_view.capture(clear_output=True)
def use_select_images_button_clicked(button):
    # Prompt the user to select a directory of images                              
    root.withdraw()                                        # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True)   # Raise the root to the top of all windows.
    root.filename =  filedialog.askdirectory(initialdir = "/samples",title = "Select directory of images (or npzs) to segment")
    # Save the filename as an attribute of the button
    if root.filename:
        sample_direc= root.filename
        model_dict["sample_direc"]=root.filename
        print(f"The images in the folder will be segmented :\n{sample_direc} ")
        jpgs = glob.glob1(sample_direc + os.sep, "*jpg")
        if jpgs == []:
            raise Exception("The directory contains no jpgs! Please select a directory with jpgs.")
    else:
        raise Exception("You must select a valid directory first!")

@run_model_view.capture(clear_output=True)
def run_model_button_clicked(button):
    print("Called Run Model")
    if model_dict["sample_direc"] is None:
        print("You must click 'Use Data' or 'Select Images' First")
        return
    else:
        if model_dict["use_GPU"] == False:
            print("Not using the GPU")
            ## to use the CPU (not recommended):
            os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
        elif model_dict["use_GPU"] == True:
            print("Using the GPU")
            ##use the first available GPU
            os.environ['CUDA_VISIBLE_DEVICES'] = '0' #'1'
        if 'MNDWI' in model_dropdown.value:
            dataset='MNDWI'
        else:
            dataset='RGB'
        # Disable the run button while the model is running
        run_model_button.disabled = True
        dataset_id=model_dict['model_type']
        # model_dict['implementation']=model_implementation.value
        model_choice=model_dict['implementation']
        zoo_model=zoo_model_module.Zoo_Model()
        # First download the specified model
        zoo_model.download_model(dataset,dataset_id)
        # Get weights as list
        Ww=zoo_model.get_weights_list(model_choice)
        # Load the model from the config files
        model, model_list, config_files, model_types=zoo_model.get_model(Ww)
        metadatadict=zoo_model.get_metadatadict(Ww,config_files, model_types)
        # # Compute the segmentation
        zoo_model.compute_segmentation(model_dict["sample_direc"], model_list, metadatadict)
        # Enable run button when model is done
        run_model_button.disabled = False
    
    
model_implementation.observe(handle_model_implementation,"value")
model_dropdown.observe(handle_model_type,"value")
GPU_checkbox.observe(handle_GPU_checkbox,"value")


run_model_button.on_click(run_model_button_clicked)
use_data_button.on_click(use_data_button_clicked)
use_select_images_button.on_click(use_select_images_button_clicked)

model_choices_box=HBox([model_dropdown, model_implementation])

line_widget=HTML(value="____________________________________________________")

instr_header=HTML(
    value="<h4>Click ONE of the following buttons:</h4>",layout=Layout(margin='0px 0px 0px 0px'))

instr_use_data=HTML(
    value="<b>1. Use Data Folder Button</b> \
        <br> - When CoastSat downloads imagery it created a folder called 'data'in the CoastSeg directory.\
            The jpgs within the 'data' folder will be copied to another folder with a name such as\
            <span style=\"background-color:LightGray;\">segmentation_data_2022-07-07__10_hr_04_min58 </span>\
            (the date and time will be the current date and time) <br> \
           The model will be applied to this folder and the model outputs will be generated within a subdirectory \
            called 'out'",layout=Layout(margin='0px 0px 0px 20px'))

instr_select_images=HTML(
    value="<b>2. Select Images Button</b> \
        <br> - This will open a pop up window where the folder containing the jpgs can be selected.<br>\
             - The model will be applied to this folder and the model outputs will be generated within a subdirectory\
             called 'out'<br>\
      - <span style=\"background-color:yellow;\">WARNING :</span> You will not be able to see the files within the folder you select.<br>\
    ",layout=Layout(margin='0px 0px 0px 20px'))

instr_run_model=HTML(
    value="<b>3. Run Model Button</b> \
        <br> - Make sure to click Select Images Button or Use Data Button.<br>\
             - The model will be applied to the selected folder and the model outputs will be generated within a subdirectory\
             called 'out'<br>\
      - <span style=\"background-color:yellow;\">WARNING :</span> You should not run multiple models on the same folder. Otherwise not all the model outputs\
      will be saved to the folder.<br>\
    ",layout=Layout(margin='0px 0px 0px 20px'))


instr_vbox=VBox([instr_header, line_widget, instr_use_data, instr_select_images, instr_run_model])

display(GPU_checkbox,GPU_view,model_choices_box,instr_vbox,use_data_button,use_select_images_button,line_widget, model_view,run_model_button,run_model_view)

Checkbox(value=False, description='Use GPU?', indent=False)

Output()

Button(description='Use Data Button', style=ButtonStyle())

Button(description='Select Your Images', style=ButtonStyle())

HTML(value='____________________________________________________')

Output(layout=Layout(border='1px solid black'))

Button(description='Run Model', style=ButtonStyle())

Output(layout=Layout(border='1px solid black'))